In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/indic-tts-deepfake-challenge/sample.csv


In [2]:
import torch
from transformers import Trainer, TrainingArguments, AutoModelForAudioClassification, Wav2Vec2Processor, DataCollatorWithPadding
from sklearn.metrics import roc_auc_score
from datasets import Dataset, load_dataset
from sklearn.model_selection import train_test_split
from transformers import AutoFeatureExtractor

In [3]:
# Load the dataset from Hugging Face
dataset = load_dataset("SherryT997/IndicTTS-Deepfake-Challenge-Data")

dataset

README.md:   0%|          | 0.00/2.81k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/35 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/35 [00:00<?, ?it/s]

train-00000-of-00035.parquet:   0%|          | 0.00/453M [00:00<?, ?B/s]

train-00001-of-00035.parquet:   0%|          | 0.00/461M [00:00<?, ?B/s]

...

test-00003-of-00004.parquet:   0%|          | 0.00/291M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/31102 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2635 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/35 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'id', 'language', 'is_tts', 'audio'],
        num_rows: 31102
    })
    test: Dataset({
        features: ['text', 'id', 'language', 'is_tts', 'audio'],
        num_rows: 2635
    })
})

In [4]:
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [5]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=16000, max_length=16000, truncation=True, padding=True
    )
    return inputs

preprocessed = dataset['train'].map(preprocess_function, remove_columns=["audio","text","id","language"], batched=True)
preprocessed = preprocessed.rename_column("is_tts", "label")

preprocessed

Map:   0%|          | 0/31102 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'input_values'],
    num_rows: 31102
})

In [6]:
# Load Wav2Vec2 Processor and Model
model = AutoModelForAudioClassification.from_pretrained("facebook/wav2vec2-base-960h", num_labels=2)

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Function to compute ROC-AUC for evaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Get probabilities for positive class (index 1)
    preds = torch.nn.functional.softmax(torch.tensor(logits), dim=1)[:, 1].numpy()
    auc = roc_auc_score(labels, preds) if len(set(labels)) > 1 else 0.5
    return {"roc_auc": auc}

In [8]:
train_eval_dataset = preprocessed.train_test_split(test_size=0.1)

train_eval_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'input_values'],
        num_rows: 27991
    })
    test: Dataset({
        features: ['label', 'input_values'],
        num_rows: 3111
    })
})

In [9]:
training_args = TrainingArguments(
    output_dir="final_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=64,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    warmup_ratio=0.05,
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="eval_roc_auc",
    report_to="none"
)
    
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_eval_dataset["train"],
    eval_dataset=train_eval_dataset["test"],
    processing_class=feature_extractor,
    compute_metrics=compute_metrics,
)

In [10]:
trainer.train()

trainer.save_model("wav2vec2_final_model")

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Roc Auc
1,0.666000,0.526728,0.858545
2,0.487400,0.262046,0.969418
3,0.316600,0.160775,0.981672
4,0.262700,0.128192,0.990316
5,0.188500,0.084634,0.995263
6,0.145900,0.076257,0.996134
7,0.123600,0.075953,0.994172
8,0.106900,0.107707,0.995668
9,0.085200,0.077966,0.996491


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

In [11]:
test = dataset['test']

feature_extractor = AutoFeatureExtractor.from_pretrained("/kaggle/working/wav2vec2_final_model")
model = AutoModelForAudioClassification.from_pretrained("/kaggle/working/wav2vec2_final_model")

# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # Set model to evaluation mode

Wav2Vec2ForSequenceClassification(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)


In [12]:
idx = []
is_tts = []

x = 1
for i in test:
    idx.append(i['id'])

    # Extract features
    inputs = feature_extractor(i["audio"]["array"], sampling_rate=16000, return_tensors="pt")
    
    # Move inputs to GPU
    inputs = {key: val.to(device) for key, val in inputs.items()}

    with torch.no_grad():
        logits = model(**inputs).logits

    # Move logits back to CPU before converting to NumPy
    softmax_values = torch.nn.functional.softmax(logits, dim=1)
    softmax_second_element = softmax_values[0, 1].item()  # Convert to Python float

    is_tts.append(softmax_second_element)

    print(x,end="\r")
    x += 1

# Create dataframe and save to CSV
df = pd.DataFrame({"id": idx, "is_tts": is_tts})

df.to_csv("submission.csv", index=False)

In [13]:
df = pd.read_csv("/kaggle/working/submission.csv")
df

,id,is_tts
0,ASM_F_ANGER_00109,0.003141
1,ASM_F_ANGER_00127,0.997384
2,ASM_F_ANGER_00386,0.003011
3,ASM_F_ANGER_00103,0.997394
4,ASM_F_ANGER_00434,0.003236
...,...,...
2630,TAM_F_SURPRISE_00387,0.997373
2631,TAM_F_SURPRISE_00173,0.997419
2632,TAM_F_SURPRISE_00718,0.003096
2633,TAM_F_SURPRISE_00127,0.003215
